In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install undetected-chromedriver

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install undetected-chromedriver pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import time
import random
import os
import pandas as pd
import undetected_chromedriver as uc
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# --- CONFIGURATION DES PARAMETRES ---

# Liste des URLs cibles fournies pour le monitoring
LISTE_URLS = [
    #Iphone 16
    "https://fr.aliexpress.com/item/1005010710620938.html",
    "https://fr.aliexpress.com/item/1005009551954092.html",

    #Iphone 16 pro max
    "https://fr.aliexpress.com/item/1005010555560222.html",
    "https://fr.aliexpress.com/item/1005010797047447.html",

    #Iphone 15
    "https://fr.aliexpress.com/item/1005010555245443.html",
    "https://fr.aliexpress.com/item/1005009439484520.html",

    #Iphone 15 pro max
    "https://fr.aliexpress.com/item/1005010568769219.html",
    "https://fr.aliexpress.com/item/1005010542816186.html",

    #Iphone 14+
    "https://fr.aliexpress.com/item/1005009516687421.html",
    "https://fr.aliexpress.com/item/1005005511011469.html",

    #Iphone 14 pro max
    "https://fr.aliexpress.com/item/1005010568832096.html",
    "https://fr.aliexpress.com/item/1005005513213583.html",

    #S24
    "https://fr.aliexpress.com/item/1005007998182446.html",
    "https://fr.aliexpress.com/item/1005006779274898.html",

    #S24 ultra
    "https://fr.aliexpress.com/item/1005007998285304.html",
    "https://fr.aliexpress.com/item/1005009541498339.html",

    #S23
    "https://fr.aliexpress.com/item/1005009871463745.html",
    "https://fr.aliexpress.com/item/1005010677218786.html",

    #S23 ultra
    "https://fr.aliexpress.com/item/1005008189086764.html",
    "https://fr.aliexpress.com/item/1005010737189750.html",

    #Pura 70 pro
    "https://fr.aliexpress.com/item/1005008886526337.html",
    "https://fr.aliexpress.com/item/1005007974212428.html",

    #Pura 70 ultra
    "https://fr.aliexpress.com/item/1005008988580304.html",
    "https://fr.aliexpress.com/item/1005008905220105.html",

    #Xiaomi 15
    "https://fr.aliexpress.com/item/1005010788926638.html",
    "https://fr.aliexpress.com/item/1005010050085154.html",

    #Xiaomi 14
    "https://fr.aliexpress.com/item/1005010291330904.html",
    "https://fr.aliexpress.com/item/1005010048067829.html",

]

EXCEL_FILE = "suivi_complet_aliexpress.xlsx"

# --- FONCTIONS TECHNIQUES ---

def get_driver():
    """Initialise le navigateur avec des mesures anti-captcha renforcees."""
    options = uc.ChromeOptions()
    
    # Anti-Bot: Taille de fenetre variable pour simuler differents ecrans
    w = random.randint(1200, 1920)
    h = random.randint(800, 1080)
    options.add_argument(f"--window-size={w},{h}")
    
    # Optimisation: On ne charge pas les images pour la rapidite et la discretion
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--blink-settings=imagesEnabled=false")
    
    # Demarrage du driver furtif
    driver = uc.Chrome(options=options, version_main=143, use_subprocess=True)
    
    # Anti-Bot: On retire la mention 'webdriver' des proprietes Javascript
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
    })
    
    return driver

def force_close_popups(driver):
    """Gere la fermeture automatique des bandeaux de cookies et publicites."""
    try:
        wait = WebDriverWait(driver, 5)
        cookie_btn = wait.until(EC.element_to_be_clickable((By.XPATH, 
            "//button[contains(.,'Accepter') or contains(.,'Accept') or contains(.,'Okay')]")))
        # Simulation d'un temps de reaction humain
        time.sleep(random.uniform(0.5, 1.5))
        cookie_btn.click()
    except:
        pass

def scrape_product(driver, url):
    """Effectue la navigation et l'extraction des donnees pour un produit donne."""
    try:
        driver.get(url)
        # Temps de lecture aleatoire pour eviter la detection de pattern
        time.sleep(random.uniform(8, 15))
        force_close_popups(driver)
        
        # Mouvement de souris aleatoire pour simuler une presence humaine
        action = ActionChains(driver)
        action.move_by_offset(random.randint(5, 50), random.randint(5, 50)).perform()
        
        now = datetime.now()
        data = {
            "date": now.strftime("%d/%m/%Y"),
            "heure": now.strftime("%H:%M:%S"),
            "nom_article": "N/A",
            "prix": "N/A",
            "vendeur_nom": "N/A",
            "vendeur_location": "N/A",
            "vendeur_date_ouverture": "N/A",
            "note_article": "N/A",
            "note_communication": "N/A",
            "note_livraison": "N/A",
            "url": url
        }

        # Extraction du titre et du prix courant
        try:
            data["nom_article"] = driver.find_element(By.XPATH, "//h1").text.strip()
            data["prix"] = driver.find_element(By.CSS_SELECTOR, "span.price-default--current--F8OlYIo").text.strip()
        except: 
            pass

        # Interaction pour reveler les details du vendeur
        try:
            # Scroll par etapes pour simuler un utilisateur reel
            for _ in range(3):
                driver.execute_script(f"window.scrollBy(0, {random.randint(100, 200)});")
                time.sleep(random.uniform(0.3, 0.6))

            # Attente de l'element 'Vendu par' pour declencher le survol
            hover_zone = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "store-detail--title--qt8UBeq")))
            
            actions = ActionChains(driver)
            actions.move_to_element(hover_zone).pause(random.uniform(1.0, 1.5)).perform()
            time.sleep(3)

            # Fonction pour extraire la valeur situee apres un libelle specifique (XPath following-sibling)
            def get_val(lbl):
                try: 
                    xpath = f"//td[contains(text(), '{lbl}')]/following-sibling::td"
                    return driver.find_element(By.XPATH, xpath).text.strip()
                except: 
                    return "N/A"

            # Recuperation des informations vendeur par libelles exacts
            data["vendeur_nom"] = get_val("Nom") or get_val("Name")
            data["vendeur_location"] = get_val("Emplacement") or get_val("Location")
            data["vendeur_date_ouverture"] = get_val("Ouvert depuis") or get_val("Open Date")
            
            # Recuperation des notes de performance par libelles exacts
            data["note_article"] = get_val("Article comme décrit")
            data["note_communication"] = get_val("communication")
            data["note_livraison"] = get_val("Délais de livraison")
            
        except: 
            pass

        return data
    except Exception as e:
        print(f"Erreur sur {url} : {e}")
        return None

def save_to_excel(new_data_list):
    """Enregistre les donnees sans ecraser l'historique existant."""
    df_new = pd.DataFrame(new_data_list)
    if os.path.exists(EXCEL_FILE):
        df_old = pd.read_excel(EXCEL_FILE)
        df_final = pd.concat([df_old, df_new], ignore_index=True)
    else:
        df_final = df_new
    df_final.to_excel(EXCEL_FILE, index=False)
    return df_final

# --- BOUCLE DE SURVEILLANCE PRINCIPALE ---

print(f"Tracker active a {datetime.now().strftime('%H:%M:%S')}. Cycle de 1h30.")

while True:
    session_start = datetime.now()
    print(f"--- SESSION DU {session_start.strftime('%d/%m/%Y a %H:%M:%S')} ---")
    
    # On regenere une instance propre a chaque session pour vider les cookies de session
    driver = get_driver()
    session_results = []

    try:
        # Ordre de visite aleatoire pour ne pas creer de pattern de navigation
        urls_random = LISTE_URLS.copy()
        random.shuffle(urls_random)

        for i, url in enumerate(urls_random):
            res = scrape_product(driver, url)
            if res:
                session_results.append(res)
                nom_court = res['nom_article'][:30] + "..." if len(res['nom_article']) > 30 else res['nom_article']
                # Affichage des resultats incluant les notes et la localisation
                print(f"   [{i+1}/{len(LISTE_URLS)}] {res['prix']} | {nom_court} | {res['vendeur_nom']} | {res['vendeur_location']}")
                print(f"      Notes: {res['note_article']} / {res['note_communication']} / {res['note_livraison']}")
            
            # Pause entre produits
            time.sleep(random.uniform(5, 12))
            
            # Pause de securite longue toutes les 10 requetes (Anti-Captcha)
            if (i + 1) % random.randint(8, 11) == 0:
                print("   Pause respiration longue (90s) pour l'IP...")
                time.sleep(random.uniform(85, 105))

        if session_results:
            save_to_excel(session_results)
            print("Sauvegarde de la session terminee.")

    except Exception as e:
        print(f"Erreur durant la session : {e}")
    finally:
        driver.quit()

    # Calcul du temps restant avant le prochain cycle de 30 minutes
    elapsed = (datetime.now() - session_start).total_seconds()
    wait_time = max(0, (5400 + random.randint(-120, 120)) - elapsed)
    
    print(f"Prochaine session prevue a : {datetime.fromtimestamp(time.time() + wait_time).strftime('%H:%M:%S')}")
    time.sleep(wait_time)

Tracker active a 20:15:05. Cycle de 1h30.
--- SESSION DU 08/02/2026 a 20:15:05 ---
